In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.misc as misc
import math
import imageio

# Project Imports
import llops as yp
import llops.operators as ops
from llops.solvers import objectivefunctions, regularizers, iterative
import llops.simulation as sim

# Set default backend and dtype
ops.setDefaultBackend('numpy') # arrayfire or numpy
ops.setDefaultDatatype('complex32')

# Generate Simple Forward Model

In [2]:
# Image size to simulate
image_size = np.array([128, 256])

# Create test object
x = sim.brain(image_size)

# Generate convolution kernel h
h_size = np.array([4, 4])
h = yp.zeros(image_size)
h[image_size[0] // 2 - h_size[0] // 2:image_size[0] // 2 + h_size[0] // 2,
  image_size[1] // 2 - h_size[1] // 2:image_size[1] // 2 + h_size[1] // 2] = yp.randn((h_size[0], h_size[1]))
h /= yp.sum(h)

# Generate Convolution Kernel
C = ops.Convolution(h)

# Generate Measurement
y = C * x

# Display Result
plt.figure()
plt.subplot(121)
plt.imshow(yp.abs(x))
plt.subplot(122)
plt.imshow(yp.abs(y))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:

# Generate two different diagonal operators
d0, d1 = yp.randn(image_size), yp.randn(image_size)
D0 = ops.Diagonalize(d0, dtype='complex32')
D1 = ops.Diagonalize(d1, dtype='complex32')
F = ops.FourierTransform(image_size)

# Combine into a single operator
A = D0 * F * D1

print(A.get_argument(D0).__array_interface__['data'])
print(d0.__array_interface__['data'])
print(A.get_argument(D1).__array_interface__['data'])
print(d1.__array_interface__['data'])

# Ensure we can get arguments
yp.assert_equality(A.get_argument(D0), d0)
yp.assert_equality(A.get_argument(D1), d1)

# Try to set d1 to d0
A.set_argument(d0, D1)
print(A.get_argument(D1).__array_interface__['data'])
print(d0.__array_interface__['data'])

(5399502848, False)
(5399502848, False)
(5485363200, False)
(5485363200, False)
(5399502848, False)
(5399502848, False)


In [1]:
from llops.operators import Operator

class ObjectiveFunction(Operator):
    """An objective function class. This class is mostly the same as an 
       llops.operators.Operator, but also supports gradients with respect 
       to multiple variables."""

    def __init__(self, operator, label='O'):

        # Check if this objective is multivariate, based on type(operator)
        if type(operator) in (list, tuple):
            self._operator_list = [op[0] for op in operator]
            self._operand_list = [op[1] for op in operator]
        else:
            self._operator_list = [operator]
            self._operand_list = None

        super().__init__(shape=self._operator_list[0].shape,
                         dtype=self._operator_list[0].dtype,
                         backend=self._operator_list[0].backend,
                         cost=self._operator_list[0].cost,
                         label=label,
                         repr_latex=self._operator_list[0].repr_latex,
                         repr_str=self._operator_list[0].repr_str,
                         convex=self._operator_list[0].convex,
                         smooth=self._operator_list[0].smooth)

    def _forward(self, *x, y=None):
        self.operator_list[0]._forward(x, y)
    
    def _adjoint(self, *x, y=None):
        self.operator_list[0]._adjoint(x, y)

    def _inverse(self, *x, y=None):
        self.operator_list[0]._inverse(x, y)

    def _gradient(self, *x, y=None):
        self.operator_list[0]._gradient(x, y)
        
    def _proximal(self, *x, y=None):
        self.operator_list[0]._proximal(x, y)

    @property
    def multivariate(self):
        return self.operand_list is not None

# Generate two different diagonal operators
d, x = yp.randn(image_size), yp.randn(image_size)
D = ops.Diagonalize(d, dtype='complex32')
X = ops.Diagonalize(x, dtype='complex32')
F = ops.FourierTransform(image_size)
L2 = ops.L2Norm(image_size)

# Combine into a single operator
Ax = L2 * (F.H * D * F)
Ad = L2 * (F.H * X * F)

operator_list = [(Ax, X), (Ad, D)]

objective = ObjectiveFunction(operator_list)


NameError: name 'yp' is not defined